In [ ]:
''' Import dependencies.

Moduels:
    tensorflow: The neural network framework.
    layer: The customized single layers.
    module: The customized multi-layer modules.
    
    os: The basic module for path parsing.
    json: The basic modue for config parsing.
    zipfile: Extract zip file.
    random: Shuffle indices.
    scipy.io: Load .mat file.
    numpy: To process .mat file data.
    
    tqdm: The 3rd-party looping visualzer.
'''

import layer, module
import tensorflow as tf
import os, json, zipfile, random
import scipy.io, scipy.misc, imageio, math, numpy as np
from enum import Enum
from tqdm import tqdm, trange
from PIL import Image

In [ ]:
''' Parsing config.

The config file is saved to root/config.
'''
CONFIG_PATH = os.path.abspath('../config.json')
with open(CONFIG_PATH) as CONFIG_FILE:
    CONFIG = json.loads(CONFIG_FILE.read())
    flags = tf.app.flags
    flags.DEFINE_string('project', CONFIG['project'], 'The project name.')
    
    flags.DEFINE_string('path_to_pretrained',
                        os.path.join(
                            os.path.expanduser(CONFIG['pretrained']['path']),
                            CONFIG['project']),
                        'The path to pretrained parameters.')
    flags.DEFINE_boolean('load_pretrained', CONFIG['pretrained']['load'], 'Load the latest pretrained parameters.')
    flags.DEFINE_boolean('save_pretrained', CONFIG['pretrained']['save'], 'Save the trained parameters.')
    
    flags.DEFINE_string('path_to_log',
                        os.path.join(
                            os.path.expanduser(CONFIG['log']['path']),
                            CONFIG['project']),
                        'The path to log.')
    flags.DEFINE_boolean('save_log', CONFIG['log']['save'], 'Save the log.')
    
    flags.DEFINE_string('path_to_data',
                        os.path.expanduser(CONFIG['data']['path']),
                        'The path to data.')
    flags.DEFINE_string('name_of_data', CONFIG['data']['name'], 'The name of data.')
    
    flags.DEFINE_string('task', CONFIG['task']['step'], 'The task to be done.')
    flags.DEFINE_integer('epoch', CONFIG['task']['train']['epoch'], 'The epoch to be trained.')
    flags.DEFINE_string('metric', CONFIG['task']['eval']['metric'], 'The evaluation metric.')
    flags.DEFINE_float('metric_coefficient', CONFIG['task']['eval']['coefficient'], 'The evaluation metric coefficient.')
    
FLAGS = flags.FLAGS

In [ ]:
'''
matlab_path = '/media/nulledge/UBUNTU 16_0/MPII/mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat'
annotation = scipy.io.loadmat(matlab_path)['RELEASE']
'''

In [ ]:
'''
img_train = annotation['img_train'][0, 0][0, :]
valid_index = []

img_idx = 24730
r_idx = 1

ref = annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]['annopoints'][0, 0]['point'][0, :]

for point in ref:
    print(point.dtype)
'''
'''
humans_in_image = self.__annotation['annolist'][0, 0][0, img_idx]['annorect'].shape[1]

for r_idx in range(humans_in_image):
    valid_index.append((img_idx, r_idx))
'''

In [3]:
''' Common metadata.
'''

class DataInterface(object):
    def __init__(self):
        if self.__corrupted():
            self.__reload()
    
    def __corrupted(self):
        raise NotImplementedError()
        
    def __reload(self):
        raise NotImplementedError()
        
    def getBatch(self, batch_size):
        return [(lambda x: 
                 (self.__getImage(x),
                 self.__getHeatmaps(x),
                 self.__getExtra(x))
                )(self.__getNext()) in range(batch_size)]
        
    def __getImage(self, index):
        raise NotImplementedError()
        
    def __getHeatmaps(self, index):
        raise NotImplementedError()
        
    def __getExtra(self, index):
        raise NotImplementedError()
        
    def __getNext(self):
        raise NotImplementedError()
        
class DataCenter(object):
    def __init__(self, root):
        self.__root = root
    
    def request(self, data, task):
        path = os.path.join(self.__root, data)
        
        if data == 'FLIC':
            return FLIC(root = path, task = task)
        elif data == 'MPII':
            return MPII(root = path, task = task)
        else:
            raise NotImplementedError()
            
class Joint(Enum):

    # FLIC
    L_Shoulder  =   1
    R_Shoulder  =   2
    L_Elbow     =   3
    R_Elbow     =   4
    L_Wrist     =   5
    R_Wrist     =   6
    L_Hip       =   7
    R_Hip       =   8
    L_Knee      =   9
    R_Knee      =   10
    L_Ankle     =   11
    R_Ankle     =   12

    L_Eye       =   13
    R_Eye       =   14
    L_Ear       =   15
    R_Ear       =   16
    M_Nose      =   17

    M_Shoulder  =   18
    M_Hip       =   19
    M_Ear       =   20
    M_Torso     =   21
    M_LUpperArm =   22
    M_RUpperArm =   23
    M_LLowerArm =   24
    M_RLowerArm =   25
    M_LUpperLeg =   26
    M_RUpperLeg =   27
    M_LLowerLeg =   28
    M_RLowerLeg =   29

    # MPII
    # M_Pelvis    =   M_Hip
    # M_Thorax    =   M_Torso
    M_UpperNeck =   30
    M_HeadTop   =   31
    
class Basis(Enum):
    x = 0
    y = 1
    
def squeeze(annotation, indices):
    if len(indices) == 0:
        return np.squeeze(annotation)
    return squeeze(np.squeeze(annotation)[indices[0]], indices[1:])

In [4]:
''' FLIC data reader
'''       
class FLIC(DataInterface):
    
    NUMBER_OF_DATA = 5003
    TRAIN_RATIO = 0.9
    EVAL_RATIO = 1.0 - TRAIN_RATIO
    
    joint2index = {
        Joint.L_Shoulder: 0,
        Joint.L_Elbow: 1,
        Joint.L_Wrist: 2,
        Joint.R_Shoulder: 3,
        Joint.R_Elbow: 4,
        Joint.R_Wrist: 5,
        Joint.L_Hip: 6,
        Joint.R_Hip: 9,
        Joint.L_Eye: 12,
        Joint.R_Eye: 13,
        Joint.M_Nose: 16
    }
    
    def __init__(self, root, task):
        self.__root = root
        
        if self.__corrupted():
            self.__reload()
            
        # self.__extract_path = os.path.join(self.__root, 'FLIC')
        self.__matlab_path = os.path.join(self.__extract_path, 'examples.mat')
        self.__annotation = scipy.io.loadmat(self.__matlab_path)['examples']
        self.__task = task
        self.__index = { 'train': open(os.path.join(self.__root, 'train.txt'), 'r'),
                       'eval': open(os.path.join(self.__root, 'eval.txt'), 'r')}
        
    def __delete__(self):
        self.__index['train'].close()
        self.__index['eval'].close()
    
    def __corrupted(self):
        self.__archive_path = os.path.join(self.__root, 'FLIC.zip')
        print('Check if the archive file exists...', end = '')
        if not os.path.exists(self.__archive_path):
            raise Exception('You have to download the archive file from https://bensapp.github.io/flic-dataset.html')
        print('success!')
        print('\tpath:', self.__archive_path)
        
        
        self.__extract_path = os.path.join(self.__root, 'FLIC')
        print('Check if the archive file is extracted...', end = '')
        if not os.path.exists(self.__extract_path):
            print('failed!')
            return True
        print('success!')
        print('\tpath:', self.__extract_path)
        
        
        self.__index_path = { 'train': os.path.join(self.__root, 'train.txt'),
                             'eval': os.path.join(self.__root, 'eval.txt')}
        print('Check if the index file is exists...', end = '')
        if not (os.path.exists(self.__index_path['train']) and os.path.exists(self.__index_path['eval'])):
            print('failed!')
            return True
        print('success!')
        print('\ttrain path:', self.__index_path['train'])
        print('\teval path:', self.__index_path['eval'])
        
        return False
        
    def __reload(self):
        # archive_path = os.path.join(self.__data_path, FLIC.archive_file)
        print('Extract the archive file...', end = '')
        with zipfile.ZipFile(self.__archive_path, 'r') as archive_ref:
            archive_ref.extractall(self.__root)
        print('success!')
        print('\tpath:', self.__extract_path)
        
        
        index_list = [i for i in range(FLIC.NUMBER_OF_DATA)]
        random.shuffle(index_list)
        print('Generate the random train/eval set...', end = '')
        with open(self.__index_path['train'], 'w') as train_index:
            for i in range(int(FLIC.TRAIN_RATIO * FLIC.NUMBER_OF_DATA)):
                train_index.write(str(index_list[i]) + '\n')
                
        with open(self.__index_path['eval'], 'w') as eval_index:
            for i in range(int(FLIC.TRAIN_RATIO * FLIC.NUMBER_OF_DATA), FLIC.NUMBER_OF_DATA):
                eval_index.write(str(index_list[i]) + '\n')
        print('success!')
        print('\ttrain path:', self.__index_path['train'])
        print('\teval path:', self.__index_path['eval'])
        
    def getBatch(self, batch_size):
        
        batch_index = []
        
        for _ in range(batch_size):
            index = self.__getNext()
            if index is None:
                break
            batch_index.append(index)
        
        batch_image = np.ndarray(shape = (len(batch_index), 256, 256, 3), dtype = np.float32)
        for index in range(len(batch_index)):
            batch_image[index][:, :, :] = self.__getImage(batch_index[index])
            
        batch_heat = np.ndarray(shape = (len(batch_index), 64, 64, len(Joint)), dtype = np.float32)
        for index in range(len(batch_index)):
            batch_heat[index][:, :, :] = self.__getHeat(batch_index[index])
            
        batch_extra = np.ndarray(shape = (len(batch_index), 1), dtype = np.float32)
        for index in range(len(batch_index)):
            batch_extra[index][:] = self.__getExtra(batch_index[index])
            
        return batch_image, batch_heat, batch_extra, FLIC.__getMasking()
        
            
    def __getNext(self):
        index = self.__index[self.__task].readline()
        if index == '':
            return None
        return int(index)
            
    def __getImage(self, index):
        torso = squeeze(self.__annotation, ['torsobox', index])
        resolution = squeeze(self.__annotation, ['imgdims', index])
        center = int((torso[2] + torso[0])/2)
        
        pad_left = 0
        pad_right = 0 
        if center + int(resolution[0]/2) > resolution[1]:
            pad_right = center + int(resolution[0]/2) - resolution[1]
        if center - int(resolution[0]/2) < 0:
            pad_left = int(resolution[0]/2) - center
            center += pad_left
        
        return scipy.misc.imresize(np.pad(
            imageio.imread(
                os.path.join(
                    self.__extract_path,
                    'images',
                    squeeze(self.__annotation, ['filepath', index]).item()
                )
            ),
            ((0, 0), (pad_left, pad_right), (0, 0)),
            'constant', constant_values = (0, 0)
        )[:, center - int(resolution[0]/2):center + int(resolution[0]/2),:], (256, 256))
    
    def __getHeat(self, index):
        heatmaps = np.ndarray(shape = (64, 64, len(Joint)))
        
        
        torso = squeeze(self.__annotation, ['torsobox', index])
        resolution = squeeze(self.__annotation, ['imgdims', index])
        center = int((torso[2] + torso[0])/2)
        
        pad_left = 0
        if center - int(resolution[0]/2) < 0:
            pad_left = int(resolution[0]/2) - center
            center += pad_left
        
        
        keypoints_ref = squeeze(self.__annotation, [index, 'coords'])
        
        for joint in Joint:
            if joint not in FLIC.joint2index:
                heatmaps[:, :, joint.value - 1] = 0
            else:
                scale = 64 / resolution[0]
                heatmaps[:, :, joint.value - 1] = 255 * FLIC.__makeGaussian(64, 1,
                      [(keypoints_ref[0][FLIC.joint2index[joint]] + pad_left - center + int(resolution[0]/2)) * scale,
                      keypoints_ref[1][FLIC.joint2index[joint]] * scale])
        return heatmaps
    
    def __makeGaussian(size, sigma = 1, center = None):

        x = np.arange(0, size, 1, np.float32)
        y = x[:,np.newaxis]

        if center is None:
            x0 = y0 = size // 2
        else:
            x0 = center[0]
            y0 = center[1]

        return 1/(sigma*np.sqrt(2*np.pi))*np.exp(-((x-x0)**2 + (y-y0)**2) / 2*sigma**2)
    
    def __getExtra(self, index):
        torso = squeeze(self.__annotation, ['torsobox', index])
        resolution = squeeze(self.__annotation, ['imgdims', index])
        return np.linalg.norm(torso[2:4] - torso[0:2]) * 64 / resolution[0]
    
    def __getMasking():
        return [(lambda joint: joint in FLIC.joint2index)(joint) for joint in Joint]

In [6]:
''' MPII data reader
'''       
class MPII(DataInterface):
    
    NUMBER_OF_DATA = -1
    TRAIN_RATIO = 0.9
    EVAL_RATIO = 1.0 - TRAIN_RATIO
    
    joint2index = {
        Joint.R_Ankle: 0,
        Joint.R_Knee: 1,
        Joint.R_Hip: 2,
        Joint.L_Hip: 3,
        Joint.L_Knee: 4,
        Joint.L_Ankle: 5,
        # Joint.M_Pelvis: 6,
        # Joint.M_Thorax: 7,
        # Joint.M_UpperNeck: 8,
        # Joint.M_HeadTop: 9,
        Joint.R_Wrist: 10,
        Joint.R_Elbow: 11,
        Joint.R_Shoulder: 12,
        Joint.L_Shoulder: 13,
        Joint.L_Elbow: 14,
        Joint.L_Wrist: 15
    }
    
    def __init__(self, root, task):
        self.__root = root
        
        if self.__corruptedExtraction():
            raise Exception('You have to extrace the archive files')
        
        print('Load the annotation file...', end = '')
        self.__matlab_path = os.path.join(self.__extract_path['annotation'], 'mpii_human_pose_v1_u12_1.mat')
        self.__annotation = scipy.io.loadmat(self.__matlab_path)['RELEASE']
        print('success!')
        
        if self.__corruptedIndex():
            self.__reloadIndex()
        
        self.__task = task
        self.__index = { 'train': open(os.path.join(self.__root, 'train.txt'), 'r'),
                       'eval': open(os.path.join(self.__root, 'eval.txt'), 'r')}
        
        
    def __delete__(self):
        self.__index['train'].close()
        self.__index['eval'].close()
        
        
    def __corruptedExtraction(self):
        self.__extract_path = { 'image': os.path.join(self.__root, 'images'),
                               'annotation': os.path.join(self.__root, 'mpii_human_pose_v1_u12_2') }
        print('Check if the archive file is extracted...', end = '')
        if not (os.path.exists(self.__extract_path['image']) and os.path.exists(self.__extract_path['annotation'])):
            raise Exception('You have to extract the archive files')
        print('success!')
        print('\timage path:', self.__extract_path['image'])
        print('\tannotation path:', self.__extract_path['annotation'])
        
        return False
    
    
    def __corruptedIndex(self):
        self.__index_path = { 'train': os.path.join(self.__root, 'train.txt'),
                             'eval': os.path.join(self.__root, 'eval.txt')}
        print('Check if the index file is exists...', end = '')
        if not (os.path.exists(self.__index_path['train']) and os.path.exists(self.__index_path['eval'])):
            print('failed!')
            return True
        print('success!')
        print('\ttrain path:', self.__index_path['train'])
        print('\teval path:', self.__index_path['eval'])
        
        return False
    
        
    def __reloadIndex(self):
        
        img_train = self.__annotation['img_train'][0, 0][0, :]
        valid_index = []

        for img_idx in range(len(img_train)):
            if img_train[img_idx] == 0:
                continue
            assert img_train[img_idx] == 1

            # humans_in_image = self.__annotation['annolist'][0, 0][0, img_idx]['annorect'].shape[1]
            humans_in_image = annotation['single_person'][0, 0][img_idx, 0]

            if humans_in_image.shape[1] == 0:
                continue

            for r_idx in humans_in_image:
                valid_index.append((img_idx, r_idx[0] - 1))

        MPII.NUMBER_OF_DATA = len(valid_index)
        
        random.shuffle(valid_index)
        
        print('Generate the random train/eval set...', end = '')
        with open(self.__index_path['train'], 'w') as train_index:
            for i in range(int(MPII.TRAIN_RATIO * MPII.NUMBER_OF_DATA)):
                train_index.write(str(valid_index[i][0]) + " " + str(valid_index[i][1]) + '\n')
                
        with open(self.__index_path['eval'], 'w') as eval_index:
            for i in range(int(MPII.TRAIN_RATIO * MPII.NUMBER_OF_DATA), MPII.NUMBER_OF_DATA):
                eval_index.write(str(valid_index[i][0]) + " " + str(valid_index[i][1]) + '\n')
        print('success!')
        print('\ttrain path:', self.__index_path['train'])
        print('\teval path:', self.__index_path['eval'])
        
    def getBatch(self, batch_size):
        
        batch_index = []
        
        for _ in range(batch_size):
            img_idx, r_idx = self.__getNext()
            if img_idx is None or r_idx is None:
                break
            batch_index.append((img_idx, r_idx))
        
        batch_image = np.ndarray(shape = (len(batch_index), 256, 256, 3), dtype = np.float32)
        for index in range(len(batch_index)):
            batch_image[index][:, :, :] = self.__getImage(batch_index[index])
            
        batch_heat = np.ndarray(shape = (len(batch_index), 64, 64, len(Joint)), dtype = np.float32)
        for index in range(len(batch_index)):
            batch_heat[index][:, :, :] = self.__getHeat(batch_index[index])
                 
        batch_extra = np.ndarray(shape = (len(batch_index), 1), dtype = np.float32)
        for index in range(len(batch_index)):
            batch_extra[index][:] = self.__getExtra(batch_index[index])
            
        return batch_image, batch_heat, batch_extra, MPII.__getMasking()
    
    def resetBatch(self):
        self.__index[self.__task].seek(0)
        
            
    def __getNext(self):
        index = self.__index[self.__task].readline()
        if index == '':
            return None, None
        img_idx, r_idx = index.split(' ')
        return int(img_idx), int(r_idx)
    
    
    def resetBatch(self):
        self.__index[self.__task].seek(0)
    
            
    def __getImage(self, index):
        img_idx, r_idx = index
        
        image_name = self.__annotation['annolist'][0, 0][0, img_idx]['image'][0, 0]['name'][0]
        image_path = os.path.join(self.__extract_path['image'], image_name)
        image_res = (int(Image.open(image_path).size[1]), int(Image.open(image_path).size[0]))
        
        center = (int(self.__annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]['objpos'][0, 0]['y'][0, 0]),
                    int(self.__annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]['objpos'][0, 0]['x'][0, 0]))
        length = int(self.__annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]['scale'][0, 0] * 200 / 2)
        
        pad_y = [int(center[0]) - length, int(center[0]) + length]
        pad_x = [int(center[1]) - length, int(center[1]) + length]
        
        if pad_y[0] >= 0:
            pad_y[0] = 0
        else:
            pad_y[0] = -pad_y[0]
            
        if pad_y[1] <= image_res[0]:
            pad_y[1] = 0
        else:
            pad_y[1] = pad_y[1] - image_res[0]
            
        if pad_x[0] >= 0:
            pad_x[0] = 0
        else:
            pad_x[0] = -pad_x[0]
            
        if pad_x[1] <= image_res[1]:
            pad_x[1] = 0
        else:
            pad_x[1] = pad_x[1] - image_res[1]
        
        return scipy.misc.imresize(
            np.pad(
                imageio.imread(
                    os.path.join(
                        self.__extract_path['image'],
                        image_name
                    )
                ),
                ((pad_y[0], pad_y[1]), (pad_x[0], pad_x[1]), (0, 0)),
                'constant', constant_values = (0, 0)
            )[center[0] + pad_y[0] - length:center[0] + pad_y[0] + length,
              center[1] + pad_x[0] - length:center[1] + pad_x[0] + length,
              :],
            (256, 256)
        )
    
    
    def __getHeat(self, index):
        heatmaps = np.ndarray(shape = (64, 64, len(Joint)))
        img_idx, r_idx = index
        
        image_name = self.__annotation['annolist'][0, 0][0, img_idx]['image'][0, 0]['name'][0]
        image_path = os.path.join(self.__extract_path['image'], image_name)
        image_res = (int(Image.open(image_path).size[1]), int(Image.open(image_path).size[0]))
        
        center = (int(self.__annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]['objpos'][0, 0]['y'][0, 0]),
                    int(self.__annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]['objpos'][0, 0]['x'][0, 0]))
        length = int(self.__annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]['scale'][0, 0] * 200 / 2)
        
        pad_y = [int(center[0]) - length, int(center[0]) + length]
        pad_x = [int(center[1]) - length, int(center[1]) + length]
        
        if pad_y[0] >= 0:
            pad_y[0] = 0
        else:
            pad_y[0] = -pad_y[0]
            
        if pad_x[0] >= 0:
            pad_x[0] = 0
        else:
            pad_x[0] = -pad_x[0]
        
        keypoints_ref = self.__annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]['annopoints'][0, 0]['point']
        
        for joint in Joint:
            heatmaps[:, :, joint.value - 1] = 0
            if joint in MPII.joint2index:
                n_joint = keypoints_ref.shape[1]
                # print('n_joint:', n_joint)
                for joint_idx in range(n_joint):
                    try:
                        visible = not (not keypoints_ref[0, joint_idx]['is_visible']
                            or keypoints_ref[0, joint_idx]['is_visible'] == '0'
                            or keypoints_ref[0, joint_idx]['is_visible'] == 0)
                    except:
                        visible = True
                    tag = keypoints_ref[0, joint_idx]['id'][0, 0]
                    
                    if MPII.joint2index[joint] != tag or visible == False:
                        continue
                        
                    scale = 64 / (length*2)
                    heatmaps[:, :, joint.value - 1] = 255 * MPII.__makeGaussian(64, 1,
                          [(int(keypoints_ref[0, joint_idx]['x'][0, 0])
                            + int(length)
                            - int(center[1])) * scale,
                          (int(keypoints_ref[0, joint_idx]['y'][0, 0]) 
                          + int(length) 
                          - int(center[0])) * scale])
                    
        return heatmaps
    
    def __makeGaussian(size, sigma = 1, center = None):

        x = np.arange(0, size, 1, np.float32)
        y = x[:,np.newaxis]

        if center is None:
            x0 = y0 = size // 2
        else:
            x0 = center[0]
            y0 = center[1]

        return 1/(sigma*np.sqrt(2*np.pi))*np.exp(-((x-x0)**2 + (y-y0)**2) / 2*sigma**2)
    
    def __getExtra(self, index):
        img_idx, r_idx = index
        
        image_name = self.__annotation['annolist'][0, 0][0, img_idx]['image'][0, 0]['name'][0]
        image_path = os.path.join(self.__extract_path['image'], image_name)
        image_res = (int(Image.open(image_path).size[1]), int(Image.open(image_path).size[0]))
        length = int(self.__annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]['scale'][0, 0] * 200)
        
        human_ref = self.__annotation['annolist'][0, 0][0, img_idx]['annorect'][0, r_idx]
        
        return np.linalg.norm(
            np.array([int(human_ref['y1'][0, 0]), int(human_ref['x1'][0, 0])])
            - np.array([int(human_ref['y2'][0, 0]), int(human_ref['x2'][0, 0])])) * 64 / length
        
    
    def __getMasking():
        return [(lambda joint: joint in MPII.joint2index)(joint) for joint in Joint]

In [7]:
dataset = DataCenter(root = FLAGS.path_to_data).request(data = FLAGS.name_of_data, task = FLAGS.task)

Check if the archive file is extracted...success!
	image path: /media/nulledge/UBUNTU 16_0/MPII/images
	annotation path: /media/nulledge/UBUNTU 16_0/MPII/mpii_human_pose_v1_u12_2
Load the annotation file...success!
Check if the index file is exists...success!
	train path: /media/nulledge/UBUNTU 16_0/MPII/train.txt
	eval path: /media/nulledge/UBUNTU 16_0/MPII/eval.txt


In [12]:
batch = dataset.getBatch(batch_size = 8)

/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:201: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:240: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:239: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [13]:
image, heatmaps, extra, mask = batch

for idx in range(8):

    imageio.imwrite('img/image' + str(idx) + '.png', image[idx])

    for x in range(64):
        for y in range(64):
            heatmaps[idx, y, x, 0] = max(heatmaps[idx, y, x, :])
    imageio.imwrite('img/heat' + str(idx) + '.png', heatmaps[idx, :, :, 0])
    print(extra[idx])
    print(mask)

/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 255.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 99.91498565673828]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 99.39185333251953]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 99.41596221923828]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/home/nulledge/Workspace

[13.353087]
[True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[13.339947]
[True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[13.340944]
[True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[13.348116]
[True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 98.6314926147461]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 247.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 98.62423706054688]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [11.0, 255.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))


[13.333333]
[True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[13.334069]
[True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[13.418282]
[True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[13.34121]
[True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 101.491943359375]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/home/nulledge/Workspace/virtual_env/tensorflow/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 101.58526611328125]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))


In [ ]:
with tf.variable_scope('input'):
    images = tf.placeholder(
        name = 'image',
        dtype = tf.float32,
        shape = [None, metadata.image.height, metadata.image.width, metadata.image.channel])
    heatmaps_groundtruth = tf.placeholder(
        name = 'heatmap_groundtruth',
        dtype = tf.float32,
        shape = [None, metadata.heatmap.height, metadata.heatmap.width, metadata.joint])
    train = tf.placeholder(
        name = 'train',
        dtype = tf.bool,
        shape = ())

In [ ]:
# input size 256 * 256 * 3

with tf.variable_scope('compress'):
    with tf.variable_scope('conv_bn_relu'):
        net = layer.conv(input = images, ksize = 7, kchannel = 64, kstride = 2) # 128 * 128 * 64
        net = layer.bn(input = net, train = train)
        net = layer.relu(input = net)

    net = module.bottleneck(input = net, kchannel = 128, train = train, name = 'A') # 128 * 128 * 128
    net = layer.pool(input = net) # 64 * 64 * 128
    net = module.bottleneck(input = net, kchannel = 128, train = train, name = 'B') # 64 * 64 * 128
    net = module.bottleneck(input = net, kchannel = 256, train = train, name = 'C') # 64 * 64 * 256

In [ ]:
class tf_Spectrum:
    Color = tf.constant([
        [128, 0, 0],
        [255, 0, 0],
        [0, 255, 0],
        [0, 255, 255],
        [0, 0, 255]
    ], dtype = tf.float64)

def tf_gray2color(gray, spectrum = tf_Spectrum.Color):
    indices = tf.floor_div(gray, 64)
    
    t = (gray - indices * 64) / (64)
    t = tf.stack([t]*3, 2)
    indices = tf.cast(indices, dtype = tf.int32)
    
    return (1-t)*tf.gather(spectrum, indices) + t*tf.gather(spectrum, indices+1)

def tf_merge(image, heatmaps):
    board = tf.zeros(
        shape = (
            metadata.heatmap.height,
            metadata.heatmap.width,
            metadata.image.channel
        ), dtype = tf.float64)
    for joint in range(metadata.joint):
        board = tf.maximum(
            board, 
            tf_gray2color(tf.cast(heatmaps[:, :, joint], dtype=tf.float64)))
    board = tf.image.resize_images(
        board, 
        [metadata.image.height, metadata.image.width])
    image = tf.cast(image, dtype = tf.float32)
    return tf.cast(tf.add(tf.multiply(board, 0.6), tf.multiply(image, 0.4)), dtype = tf.uint8)

In [ ]:
last_stage = 8
heatmaps = []

for stage in range(1, last_stage+1):
    with tf.variable_scope('hourglass_' + str(stage)):
        prev = tf.identity(net)
        net = module.hourglass(input = net, train = train) # 64 * 64 * 256

        with tf.variable_scope('inter_hourglasses'):
            net = module.bottleneck(input = net, train = train) # 64 * 64 * 256
            net = layer.conv(input = net, ksize = 1, kchannel = 256) # 64 * 64 * 256
            net = layer.bn(input = net, train = train)
            net = layer.relu(input = net)

        with tf.variable_scope('heatmap'):
            heatmap = layer.conv(input = net, ksize = 1, kchannel = metadata.joint) # 64 * 64 * joint
            heatmaps.append(heatmap)

        if stage != last_stage:
            net = layer.conv(input = net, ksize = 1, kchannel = 256, name = 'inter')\
                + layer.conv(input = heatmap, ksize = 1, kchannel = 256, name = 'heatmap')\
                + prev # 64 * 64 * 256

merged = tf_merge(images[0], heatmaps[-1][0])

In [ ]:
if FLAGS.train :
    with tf.variable_scope('loss'):
        loss = tf.losses.mean_squared_error(heatmaps_groundtruth, heatmaps[0])
        for stage in range(1, last_stage):
            loss = loss + tf.losses.mean_squared_error(heatmaps_groundtruth, heatmaps[stage])
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.AdamOptimizer(name = 'optimizer', learning_rate = 0.00025).minimize(loss)

In [ ]:
sess = tf.Session()
saver = tf.train.Saver()
reader = Reader(train = os.path.expanduser('~/Temp/train19122.dat'),
                test = os.path.expanduser('~/Temp/test2125.dat'))
if FLAGS.train:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, FLAGS.ckpt)
else:
    saver.restore(sess, FLAGS.ckpt)

In [ ]:
if FLAGS.train == True:
    for epoch in range(100):
        train_iter = tqdm_notebook(total = 19122, desc = 'epoch: ' + str(epoch) + '/100')
        for i in range(3187):
            train_images, train_heatmaps = reader.batch(size = FLAGS.batch, is_train = True)
            _, result = sess.run([optimizer, loss],
                feed_dict = {
                    images: train_images,
                    heatmaps_groundtruth: train_heatmaps,
                    train: True})
            train_iter.set_postfix(loss = result)
            train_iter.update(FLAGS.batch)
        train_iter.close();
        temp = saver.save(sess, FLAGS.ckpt)
    '''
    for epoch in tqdm_notebook(tqdm(range(10), desc = 'epoch')):
        inner_iter = tqdm_notebook(tqdm(range(563), desc = 'iter'), leave = False)
        for iterator in inner_iter:
            train_images, train_heatmaps = reader.batch(size = flag.batch_size, is_train = True)
            _, result = sess.run([optimizer, loss],
                feed_dict = {
                    images: train_images,
                    heatmaps_groundtruth: train_heatmaps,
                    train: True})
            wrap = lambda label, value: label + '(' + str(value) + ')'
            inner_iter.set_postfix(loss = result)
        temp = saver.save(sess, os.path.join(ckpt_path, 'hourglass_' + str(start_epoch + epoch + 1) + '.ckpt'))
        print("Model saved in file: %s" % temp)
        '''
else:
    heatmap_idx = 0
    total_result = []
    test_iter = tqdm_notebook(total = 2125, desc = 'test')
    for i in range(2125 // 5):
        test_images, test_heatmaps = reader.batch(size = 5, is_train = False)
        result, output_heatmap = sess.run([loss, merged],
            feed_dict = {
                images: test_images,
                heatmaps_groundtruth: test_heatmaps,
                train: False})
        test_iter.update(5)
        total_result.append(result)
        cv2.imwrite('merged' + str(heatmap_idx) + '.jpg', output_heatmap)
        heatmap_idx += 1
    test_iter.close();
    print(sum(total_result)/(2125//5))

In [ ]:
heatmap_idx = 0
total_result = []
test_iter = tqdm_notebook(total = 2125, desc = 'test')
for i in range(2125 // 5):
    test_images, test_heatmaps = reader.batch(size = 5, is_train = False)
    result, output_heatmap = sess.run([loss, merged],
        feed_dict = {
            images: test_images,
            heatmaps_groundtruth: test_heatmaps,
            train: False})
    test_iter.update(5)
    total_result.append(result)
    cv2.imwrite('merged' + str(heatmap_idx) + '.jpg', output_heatmap)
    heatmap_idx += 1
test_iter.close();
print(sum(total_result)/(2125//5))